In [5]:
import os
import shutil
import random



def split_dataset(original_dir, output_dir, split_ratio=0.8):
    classes = ['buffalo', 'cow']

    for class_name in classes:
        class_path = os.path.join(original_dir, class_name)
        if not os.path.isdir(class_path):
            print(f"⚠️ Skipping missing class folder: {class_path}")
            continue

        for breed in os.listdir(class_path):
            breed_path = os.path.join(class_path, breed)
            if not os.path.isdir(breed_path):
                continue

            images = [f for f in os.listdir(breed_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            if not images:
                continue

            random.shuffle(images)
            split_index = int(len(images) * split_ratio)

            for phase in ['train', 'val']:
                dest_dir = os.path.join(output_dir, phase, class_name, breed)
                os.makedirs(dest_dir, exist_ok=True)

            for i, img in enumerate(images):
                phase = 'train' if i < split_index else 'val'
                src_path = os.path.join(breed_path, img)
                dest_path = os.path.join(output_dir, phase, class_name, breed, img)
                shutil.copy2(src_path, dest_path)

    print(" Dataset split completed successfully!")

#  Call the function with correct Windows paths
split_dataset(
    r"D:\cattle\Cow_VS_Buffalo",    # Original dataset folder
    r"D:\cattle\dataset_split",     # Output folder for train/val
    split_ratio=0.8                 # 80% train, 20% val
)


✅ Dataset split completed successfully!


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import layers, models
import os

# Path to split dataset
dataset_dir = r"D:\cattle\dataset_split"

# Parameters
img_size = (224, 224)
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE

# Load datasets
train_ds = image_dataset_from_directory(
    os.path.join(dataset_dir, "train"),
    labels="inferred",
    label_mode="int",  # integer labels for binary classification (0 or 1)
    image_size=img_size,
    batch_size=batch_size,
    shuffle=True
)

val_ds = image_dataset_from_directory(
    os.path.join(dataset_dir, "val"),
    labels="inferred",
    label_mode="int",
    image_size=img_size,
    batch_size=batch_size
)

# Improve performance
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

# Data Augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

# Base model (pretrained MobileNetV2)
base_model = tf.keras.applications.MobileNetV2(
    input_shape=img_size + (3,),
    include_top=False,
    weights="imagenet"
)
base_model.trainable = False  # ❌ Don't fine-tune deeper layers

# Build model
inputs = layers.Input(shape=img_size + (3,))
x = data_augmentation(inputs)
x = tf.keras.applications.mobilenet_v2.preprocess_input(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)  # Binary output

model = tf.keras.Model(inputs, outputs)

# Compile
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Train
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

# Save model (both formats)
model.save("buffalo_vs_cow_model.h5")       # HDF5 format
model.save("buffalo_vs_cow_model.keras")    # Recommended format
print(" Model saved.")




# # Optionally fine-tune deeper layers
# base_model.trainable = True
# model.compile(
#     optimizer=tf.keras.optimizers.Adam(1e-5),  # Lower LR
#     loss='binary_crossentropy',
#     metrics=['accuracy']
# )
# fine_tune_epochs = 5
# model.fit(train_ds, validation_data=val_ds, epochs=fine_tune_epochs)

Found 4724 files belonging to 2 classes.
Found 1202 files belonging to 2 classes.
Epoch 1/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 194s 1s/step - accuracy: 0.6472 - loss: 0.6650 - val_accuracy: 0.8270 - val_loss: 0.4354
Epoch 2/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 178s 1s/step - accuracy: 0.8235 - loss: 0.4282 - val_accuracy: 0.8569 - val_loss: 0.3644
Epoch 3/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 184s 1s/step - accuracy: 0.8504 - loss: 0.3515 - val_accuracy: 0.8702 - val_loss: 0.3193
Epoch 4/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 181s 1s/step - accuracy: 0.8724 - loss: 0.3108 - val_accuracy: 0.8860 - val_loss: 0.2901
Epoch 5/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 180s 1s/step - accuracy: 0.8826 - loss: 0.2847 - val_accuracy: 0.8968 - val_loss: 0.2705
Epoch 6/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 185s 1s/step - accuracy: 0.9028 - loss: 0.2506 - val_accuracy: 0.9052 - val_loss: 0.2568
Epoch 7/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 208s 1s/step - accuracy: 0.9069 - loss: 0.2356 - val_accuracy: 0.9126 - val_loss: 0.2462
Epoch 8/10
148/

✅ Model saved.


In [ ]:
how to use the model

In [3]:
from tensorflow.keras.preprocessing import image
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import layers, models
import os


def predict_image(path, model):
    img = image.load_img(path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    # img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)    //////  this convertion loses the data and give wrong pridiction
    img_array = tf.expand_dims(img_array, 0)
    prediction = model.predict(img_array)[0][0]
    return "buffalo" if prediction < 0.5 else "cow"

# Example:
model = tf.keras.models.load_model("buffalo_vs_cow_model.keras")
print(predict_image("nagpuri.png", model))


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
buffalo


In [ ]:
model.summary()